In [11]:
from matplotlib import pyplot as plt
# multi-step encoder-decoder lstm 
import tensorflow as tf
from numpy import array
from tensorflow.keras import layers as L
from tensorflow.keras import models as M

size_grid = 24
size_sample = 30 #input size + predict output size(1)  (timesteps)
#Let's define size_batch to make batches with the fixed size
size_batch = 10


#Load the pre dataset
total_video = []
one_video = []  #human grid pixel change data
one_case = []
point_changes = []  #temporal pixel changes place for one video
for i in range(size_grid):  
	point_changes.append([])

video_index = -1
f = open('clap2.txt', mode = 'rt')

count_sample = 0
current_frame = 0

# size_sample frame per one clap(assumption) 15 for input 1 for output
while True:
	line = f.readline()
    #for the last point_changes, one last for and then break while
	if not line: 
		if current_frame >= size_sample :
			print('--%dth video get, samples : %d / total samples : %d--'%(video_index-1,len(one_video), count_sample))            
			total_video.append(one_video)
		break        
	numbers = line.split(' ')  #1 1 : 0 0 0 0 0 0 0 (ex)
    
	if video_index != int(numbers[0]) :
		video_index = int(numbers[0])
		#if the previous frame index is same or bigger than size_sample, there's at least one case to one video
		if current_frame >= size_sample : 
			print('--%dth video get, samples : %d / total samples : %d--'%(video_index-1,len(one_video), count_sample))
			total_video.append(one_video)   #add samples to one video sample set
		for k in range(size_grid):
			point_changes[k] = []  #reset (new video has begun)  
		one_video = []  #reset (new video has begun)            
            
    #add new value
	for i in range(size_grid):      
		val_int = int(numbers[i+3])        
		point_changes[i].append(val_int)
        
    #add new sample if the number of values is same of bigger than size_sample
	current_frame = int(numbers[1])       
	if current_frame < size_sample :
		continue
	else:
		start = current_frame - size_sample        
		#recent ones for one_case         
		for i in range(size_sample):  # ex) one_case = ((0,0,1,1,0,1), (0,0,2,2,0,2), ... )
			temp = []
			for k in range(size_grid):   # ex) (0, 0 ,1, 1, 0, 1)
				temp.append(point_changes[k][start + i])                  
			one_case.append(temp)   #one case with 20 time stamps
            
		one_video.append(one_case) #total cases               
		one_case = []        
		count_sample += 1         
          
f.close()
print("-------------------Dataset load finished----------------------")
print("-------------Total : %d Samples from %d Videos----------------"%(count_sample, len(total_video)))

--1th video get, samples : 46 / total samples : 46--
--2th video get, samples : 47 / total samples : 93--
--3th video get, samples : 57 / total samples : 150--
--7th video get, samples : 48 / total samples : 198--
--8th video get, samples : 44 / total samples : 242--
--9th video get, samples : 35 / total samples : 277--
--10th video get, samples : 24 / total samples : 301--
--11th video get, samples : 18 / total samples : 319--
--12th video get, samples : 35 / total samples : 354--
--13th video get, samples : 35 / total samples : 389--
--14th video get, samples : 48 / total samples : 437--
--15th video get, samples : 48 / total samples : 485--
--16th video get, samples : 42 / total samples : 527--
--17th video get, samples : 31 / total samples : 558--
--20th video get, samples : 37 / total samples : 595--
--21th video get, samples : 31 / total samples : 626--
--22th video get, samples : 34 / total samples : 660--
--23th video get, samples : 15 / total samples : 675--
--24th video get, 

In [12]:
import time
import datetime    #second to hour

t_steps = size_sample - 1

# define model
#input (timesteps, features)
model = tf.keras.Sequential()
model.add(L.LSTM(100, activation='relu', batch_input_shape=(size_batch, t_steps, size_grid)))
model.add(L.Dense(1)) #number of prediction
model.compile(optimizer='adam', loss='mse')

X = []
X_list = []
y = []
y_list = []

#train start
startTime = time.time()

#define dataset
#batch size = all cases from videos
for v_i in range(len(total_video)):
	print("%dth video"%int(v_i+1))
	if size_batch <= len(total_video[v_i]): 
		n_batch = int(len(total_video[v_i])/size_batch)
		print("%d batches generated"%n_batch)
                      
		for i in range(n_batch):                      
			for k in range(size_batch):                      
				# X   ex) 20 size_sample, 19 timesteps for X and last one for Y
				inputs = []
				for j in range(t_steps):         
					inputs.append(total_video[v_i][i*size_batch + k][j]) #v_i th video, ith case, jth timestep                                       
				X_list.append(inputs)
				# y
				y_list.append(total_video[v_i][i*size_batch + k][t_steps]) #v_i th video, ith case          
			X = array(X_list)
			y = array(y_list)
           
			#reshape
			X.reshape((size_batch, t_steps, size_grid))            

			# fit model
			model.fit(X, y, batch_size=size_batch, epochs=100, steps_per_epoch=size_sample, verbose=0)
    
			#flush  
			X = [] 
			X_list = []
			y = []
			y_list = []    
    

endTime = time.time() - startTime
hourtime = str(datetime.timedelta(seconds = endTime))
print("====== " + hourtime + " ======")
    
    
#and save(keras, with 'save', architecture and model are saved simultaneously)
name = str('C:/Users/zoclz/models/2finalmodel2.h5')
model.save(name)

    
print("------------All train finished---------------")

1th video
4 batches generated
2th video
4 batches generated
3th video
5 batches generated
4th video
4 batches generated
5th video
4 batches generated
6th video
3 batches generated
7th video
2 batches generated
8th video
1 batches generated
9th video
3 batches generated
10th video
3 batches generated
11th video
4 batches generated
12th video
4 batches generated
13th video
4 batches generated
14th video
3 batches generated
15th video
3 batches generated
16th video
3 batches generated
17th video
3 batches generated
18th video
1 batches generated
19th video
3 batches generated
20th video
2 batches generated
21th video
22th video
23th video
2 batches generated
24th video
4 batches generated
25th video
26th video
27th video
1 batches generated
28th video
29th video
2 batches generated
30th video
4 batches generated
31th video
3 batches generated
32th video
2 batches generated
33th video
6 batches generated
34th video
4 batches generated
35th video
12 batches generated
36th video
12 batches g